In [15]:
# !pip install pycalphad
# !pip install numpy
# !pip install tqdm
# !pip install ipywidgets

In [ ]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

In [ ]:
# TODO: fix ordered_phases (check B2_BCC and BCC_B2)
def filterOrderedPhases(dbf, candidate_phases=None):
    if candidate_phases == None:
        candidate_phases = dbf.phases.keys()
    else:
        candidate_phases = set(
            candidate_phases).intersection(dbf.phases.keys())
    # all_phases = [{"name": phase, "ordered": dbf.phases[phase].model_hints.get('ordered_phase'),'dis':dbf.phases[phase].model_hints.get('disordered_phase')} for phase in candidate_phases if dbf.phases[phase].model_hints.get('ordered_phase') != None]
    # disordered_phases = [dbf.phases[phase].model_hints.get('disordered_phase') for phase in candidate_phases]
    ordered_phases = [dbf.phases[phase].model_hints.get(
        'ordered_phase') for phase in candidate_phases]
    # disordered_phases = [dbf.phases[phase].model_hints.get('disordered_phase') for phase in candidate_phases if phase in ordered_phases]
    phases = [phase for phase in candidate_phases if
              (phase not in ordered_phases)]
    #  or (phase in disordered_phases and
    # dbf.phases[phase].model_hints.get('ordered_phase') not in candidate_phases)
    return sorted(phases)

In [ ]:
from pycalphad.core.utils import filter_phases, unpack_components

def getPossiblePhases(db, components):
    comps = list(components)
    if 'VA' not in comps:
        comps.append('VA')
    comps = unpack_components(db, comps)
    return filter_phases(db, comps)

T = 800
components = ['ZR', 'MO', 'W', 'TA', 'V', 'CR', 'CO', 'NI']
components.append('VA')
possible_phases = getPossiblePhases(db, components)
possible_phases = filterOrderedPhases(db, possible_phases)
print(len(possible_phases))

124


In [17]:
from pycalphad import calculate
def filterX(x_result):
    for x in x_result:
        if x > 0.35 or x < 0.05:
            return False
    return True
        

results = [None] * len(all_phases)
gm_result = 0
# for i in tqdm(range(len(all_phases))):
for i in range(len(all_phases)):
    if all_phases[i] in possible_phases:
            results = calculate(db, components, all_phases[i], output = 'GM', P = P, T =T)
            X_results = results.X[0][0][0]
            if len(X_results) > 1000000:
                print(f'Skipped {all_phases[i]} - {len(X_results)}')
                continue
            filter_arr = []
            for j in range(len(X_results)):
                filter_arr.append(filterX(X_results[j]))
                # if filterX(X_results[j]):
                #      print(f'{all_phases[i]} - {j}')
            
            X_results = X_results[filter_arr]
            if len(X_results) > 0:
                print(f'Added {all_phases[i]} - {X_results}')
                # gm_result = len(results.GM[0][0][0])

Skipped LAVES_C14 - 4060064
Skipped LAVES_C15 - 4060064
Skipped LAVES_C36 - 2376049
Added LIQUID - <xarray.DataArray 'X' (points: 379, component: 8)>
array([[0.104293  , 0.06100749, 0.13786785, ..., 0.09314249, 0.26099327,
        0.09657526],
       [0.0675654 , 0.05229959, 0.09682471, ..., 0.25713181, 0.1679824 ,
        0.06680157],
       [0.12246138, 0.07959303, 0.0785588 , ..., 0.15590898, 0.2051034 ,
        0.18251979],
       ...,
       [0.2240041 , 0.11481669, 0.05237775, ..., 0.07561585, 0.28192032,
        0.13673212],
       [0.05334979, 0.16287333, 0.07026131, ..., 0.12428664, 0.0564865 ,
        0.28198324],
       [0.08213114, 0.08592273, 0.06505728, ..., 0.28215556, 0.0853943 ,
        0.09550717]])
Coordinates:
  * component  (component) <U2 'CO' 'CR' 'MO' 'NI' 'TA' 'V' 'W' 'ZR'
    N          float64 1.0
    P          float64 1.013e+05
    T          float64 800.0
Dimensions without coordinates: points
Skipped MU_PHASE - 8032090
Skipped SIGMA - 2274048
